In [29]:
import pandas as pd 
import numpy as np

In [2]:
aa2 = pd.read_csv("aa2.csv") 

In [93]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [16]:
PC1=aa2.iloc[:,1].values

In [70]:
#X = array([[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60]])
#y = array([40, 50, 60, 70])
x=[]
y=[]
for i in range(0,70):
    x.append([])
    y.append([])
    x[i].append(PC1[(1+i*100):(96+i*100)])
    y[i].append(PC1[(96+i*100):(101+i*100)])
x2 = np.array(x).reshape(70,95) 
y2 = np.array(y).reshape(70,5) 

# Multilayer Perceptron model for Time Series Forecasting

In [138]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=95))
model.add(Dense(5))##dimension of output
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(x2, y2, epochs=2000, verbose=0)
# demonstrate prediction
#x_input = array([50, 60, 70])
#x_input = x_input.reshape((1, 3))
#yhat = model.predict(x_input, verbose=0)
#print(yhat)

In [91]:
RMSEtotal = 0
for i in range(70,72):
    x_input = PC1[(1+i*100):(96+i*100)]
    x_input = x_input.reshape((1, 95))
    yhat = model.predict(x_input, verbose=0)
    yreal = PC1[(96+i*100):(101+i*100)]
    RMSEtotal +=np.sqrt(np.mean(np.square(yhat-yreal)))/5
RMSEtotal/2

1.2662665905460777

In [92]:
print(yhat,yreal)

[[-5.822178  -0.6299142 -2.4337542  2.7980766  1.1447377]] [ 0.6708772 -2.012973   2.355544   9.536568  -0.6568378]


# Convolutional Neural Network model

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [99]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
x3 = x2.reshape((x2.shape[0], x2.shape[1], 1))
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(95, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(5))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(x3, y2, epochs=1000, verbose=0)

In [100]:
# demonstrate prediction
RMSEtotal = 0
for i in range(70,72):
    x_input = PC1[(1+i*100):(96+i*100)]
    x_input = x_input.reshape((1, 95, 1))
    yhat = model.predict(x_input, verbose=0)
    yreal = PC1[(96+i*100):(101+i*100)]
    RMSEtotal +=np.sqrt(np.mean(np.square(yhat-yreal)))/5
RMSEtotal/2

1.3082170421132409

In [101]:
print(yhat,yreal)

[[-5.538685   -0.18632394 -4.3196282  -1.7140886   0.3033581 ]] [ 0.6708772 -2.012973   2.355544   9.536568  -0.6568378]


# Long Short-Term Memory Neural Network model

In [103]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [106]:
#use x3
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(95, 1)))
model.add(Dense(5))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(x3, y2, epochs=1000, verbose=0)
## this one takes long time and expensive to train

In [107]:
RMSEtotal = 0
for i in range(70,72):
    x_input = PC1[(1+i*100):(96+i*100)]
    x_input = x_input.reshape((1, 95, 1))
    yhat = model.predict(x_input, verbose=0)
    yreal = PC1[(96+i*100):(101+i*100)]
    RMSEtotal +=np.sqrt(np.mean(np.square(yhat-yreal)))/5
RMSEtotal/2

1.0740439427160293

In [108]:
print(yhat,yreal)

[[-0.39466757  0.01985978  0.09555702  0.15525056  0.03969331]] [ 0.6708772 -2.012973   2.355544   9.536568  -0.6568378]


# CNN-LSTM

In [109]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [111]:
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
#sample=70, timestep=95, features=1, 
#now define subseqences to be 5, then timestep = 19
x4 = x2.reshape((x2.shape[0], 5, 19, 1))

In [ ]:
#TimeDistributedDense applies a same dense to every time step during GRU/LSTM Cell unrolling. 
#So the error function will be between predicted label sequence and the actual label sequence. 
#(Which is normally the requirement for sequence to sequence labeling problems).
#However, with return_sequences=False, Dense layer is applied only once at the last cell. 
#This is normally the case when RNNs are used for classification problem. 
#If return_sequences=True then Dense layer is applied to every timestep just like TimeDistributedDense.

In [113]:
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, 19, 1)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(5))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(x4, y2, epochs=500, verbose=0)
## this one takes long time and expensive to train

In [114]:
RMSEtotal = 0
for i in range(70,72):
    x_input = PC1[(1+i*100):(96+i*100)]
    x_input = x_input.reshape((1, 5, 19, 1))
    yhat = model.predict(x_input, verbose=0)
    yreal = PC1[(96+i*100):(101+i*100)]
    RMSEtotal +=np.sqrt(np.mean(np.square(yhat-yreal)))/5
RMSEtotal/2

1.4978600630911831

In [115]:
print(yhat,yreal)

[[ 1.3918232  3.4361594 -0.693788  -0.5503672  2.9316332]] [ 0.6708772 -2.012973   2.355544   9.536568  -0.6568378]


# Encoder-Decoder LSTM

In [116]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [142]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
#use x3
y3=y2.reshape((y2.shape[0], y2.shape[1], 1))
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(95, 1)))
model.add(RepeatVector(5))##timestep of y3 is 5
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))##1 again, because repeatvector is 5. 5*1=5
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(x3, y3, epochs=100, verbose=0)

In [143]:
RMSEtotal = 0
for i in range(70,72):
    x_input = PC1[(1+i*100):(96+i*100)]
    x_input = x_input.reshape((1, 95, 1))
    yhat = model.predict(x_input, verbose=0)
    yreal = PC1[(96+i*100):(101+i*100)]
    RMSEtotal +=np.sqrt(np.mean(np.square(yhat-yreal)))/5
RMSEtotal/2

1.0662636310115499

In [144]:
print(yhat,yreal)

[[[ 0.01487096]
  [ 0.02832061]
  [ 0.02220899]
  [-0.00614311]
  [-0.06218904]]] [ 0.6708772 -2.012973   2.355544   9.536568  -0.6568378]
